### Import mix module 

In [ ]:
import numpy as np
import pandas as pd
from CODECbreakCode.AudioMixer import SingleFileAudioMixer
import CODECbreakCode.Evaluator as Evaluator
from CODECbreakCode.Evaluator import MeasureHAAQIOutput
import CODECbreakCode.NoiseEval as NEUtil
from Optimiser.config import get_config, normalize_action, denormalize_action
import argparse

In [2]:
import warnings
warnings.filterwarnings("ignore", "Possible clipped samples in output.")
warnings.filterwarnings("ignore",message="Warning: input samples dtype is np.float64. Converting to np.float32")

In [3]:
Mixing_Path = '/home/codecrack/CodecBreakerwithRL/AudioEX/'
Noise_Generator_MP3 = SingleFileAudioMixer(filename="reggae.wav", foldpath=Mixing_Path, TrackType = NEUtil.MixingType.File)
Referece_File,_ = Noise_Generator_MP3.TestNoisedOnlyFile([0,0,0,0],"Reference.wav")
Referece_File
Referece_MP3File = Evaluator.Mp3LameLossyCompress(Referece_File,64)
print(f"Referece_MP3File:{Referece_MP3File}")

Audio duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Referece_MP3File:/home/codecrack/CodecBreakerwithRL/AudioEX/Mixing_Result_Mp3_Wav/Reference_64kbps.wav


In [4]:
####initalise the Haaqi
MeasureHAAQI = MeasureHAAQIOutput(Referece_MP3File)#Initilize the HAAQI with a permanent reference
MeasureHAAQI.MeasureHAQQIOutput(Referece_MP3File) #Test on how far from itself to itself

0.9985486850772456

In [16]:
denormalize_action(np.array([-3.604144,0.5862619,-1.3705522,-1.853153]))

array([-30. ,   8.1,   1. , 100. ])

In [5]:
def haaqi_reward_fn(solution: np.ndarray, is_normalised = True) -> float:
    if is_normalised:
        solution = denormalize_action(solution)
    #print(f'solution:{solution}')
    thre, ratio, attack, release = solution
    degradated_filename = f'audio_mixing_reggea_dynC_thre_{thre}_ratio_{ratio}_attk_{attack}_rele_{release}.wav'
    gener_Audio = Noise_Generator_MP3.TestNoisedOnlyFileOnlyDynamicCompressor(outputfilename = degradated_filename,
                                                               threshold = thre,
                                                               ratio = ratio,
                                                               attack = attack,
                                                               release = release)
    gener_Audio_mp3 = Evaluator.Mp3LameLossyCompress(gener_Audio,64)
    score = MeasureHAAQI.MeasureHAQQIOutput(gener_Audio_mp3)
#    print(f"Current degraded thre:{thre}, ratio: {ratio}, attack:{attack},release:{release} bring the HAAQI score to {score}")
    return round(1-score,3)

In [19]:
###inital
haaqi_reward_fn(np.array([-30. ,   8.1,   1. , 100.]),False)

0.252

In [ ]:
import numpy as np
m1=np.array([-0.5,-0.5,-0.5,-0.5])
m2=np.array([0.5,0.5,0.5,0.5])

def f(x):
    return -np.sum(np.log(np.sum((x-m1)**2)+0.00001)-np.log(np.sum((x-m2)**2)+0.01))

## Ranking

### function

In [ ]:
######ranking corresspoding funciton 
def haaqi_reward_fn(solution: np.ndarray) -> float:
    thre = solution[0]
    thre = thre - 15
    ratio = solution[1]
    ratio = ratio + 5
    attack = solution[2]
    attack = attack + 10
    release = solution[3] 
    release = release + 300
    degradated_filename = f'audio_mixing_reggea_dynC_thre_{thre}_ratio_{ratio}_attk_{attack}_rele_{release}.wav'
    gener_Audio = Noise_Generator_MP3.TestNoisedOnlyFileOnlyDynamicCompressor(outputfilename = degradated_filename,
                                                               threshold = thre,
                                                               ratio = ratio,
                                                               attack = attack,
                                                               release = release)
    gener_Audio_mp3 = Evaluator.Mp3LameLossyCompress(gener_Audio,64)
    score = MeasureHAAQI.MeasureHAQQIOutput(gener_Audio_mp3)
#    print(f"Current degraded thre:{thre}, ratio: {ratio}, attack:{attack},release:{release} bring the HAAQI score to {score}")
    return 1-score

### GA

In [ ]:
###Worst result for GA Until the 11.July
###haaqi_reward_fn(np.array([-12.4, 2.7, -6.8, -56]))
####Worst result for GA Until the 12.July 
haaqi_reward_fn(np.array([-14.9, 3.3, -9.0, -193]))

### RL in Continous

In [ ]:
####Worst result for GA Until the 12.July [-1.2019364  -5.0801964  -3.631278  -54.986313]
haaqi_reward_fn(np.array([-1.2019,-5.0801,-3.6313,-54.9863]))

### File Clear

In [15]:
Noise_Generator_MP3.EraseTheMp3Mixing()

### Import Discrete RL module

In [ ]:
import Optimiser

In [ ]:
import Optimiser.discrete_RL_train
from Optimiser.discrete_RL_train import discrete_RL_train as RLTrain

In [ ]:
trainner = RLTrain()
trainner.set_environments(haaqi_reward_fn)
trainner.train()

In [ ]:
###Worst result for discrete ReinforcementL Until the 11.July
haaqi_reward_fn(np.array([ -3.9 ,  -0.5 ,  -4.4 , -52.72]))

### Import Continous RL

In [7]:
import Optimiser

In [8]:
import Optimiser.continous_RL_train
from Optimiser.continous_RL_train import continous_RL_train as CRLTrain

2025-07-16 13:19:07.524523: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-16 13:19:07.557754: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-16 13:19:07.557783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-16 13:19:07.559719: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-16 13:19:07.568555: I tensorflow/core/platform/cpu_feature_guar

In [9]:
trainner = CRLTrain()
trainner.set_environments(haaqi_reward_fn)
trainner.train()

number of sub_episodes used for a single param update: 6
train_env.batch_size = parallel environment number =  1
update_num: 30, eval_intv: 3
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=True)` instead.
[step   0] mean entropy per-dim = 5.5844
final reward before udpate: -1000000000.0
final reward after udpate: 0.249
updated final_solution= [-30.  10.   1. 100.]
train_step no.= 1
best_solution of this generation= [-30.  10.   1. 100.]
best step reward= 0.249
avg step reward= 0.07745834
act_std: [-4.000e-01  9.900e+00  1.980e+01  4.948e+02]
act_mean: [-15.    5.5  10.5 300. ]
obs_mean: [-23.1   5.9   4.6 373.2]
obs_std: [  0.  10.  20. 500.]
best_step_index: [5, 2]
 
[step   1] mean entropy per-dim = 5.5884
[step   2] mean entropy per-dim = 5.5695
[step   3] mean entropy per-dim = 5.5934
train_step no.= 4
best_solution of this generation= [-30.  10.   1. 100.]
best step reward= 0.249
avg step reward= 0.075333335
act_std: [-4.000e-01  9.900e+00  1.980e+01  4.

In [36]:
RL_data, RL_F_data = trainner.save_results(filefold=Mixing_Path, para_columns=['Thre','Ratio','Attk','Release'],is_outputfulldata = False)

UnboundLocalError: local variable 'RL_Data_Full' referenced before assignment

In [ ]:
final_reward_list

In [ ]:
REINFORCE_logs

### Import GA module

In [10]:
import Optimiser.genetic_evo
from Optimiser.genetic_evo import GeneticOptimiser

In [11]:
GA_Opt = GeneticOptimiser()

In [12]:
GA_Opt.ga_init_env()

In [13]:
GA_Opt.set_fitnessfun(haaqi_reward_fn)

In [14]:
GA_Opt.run()

sol_per_pop: 30, num_genes: 4, num_generations: 30, preset_population: [[0.0, 0.0, 0.0, 0.0]]


/root/anaconda3/envs/codecbreak/lib/python3.10/site-packages/pygad/pygad.py:753: UserWarning: The percentage of genes to mutate (mutation_percent_genes=0.3) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
/root/anaconda3/envs/codecbreak/lib/python3.10/site-packages/pygad/pygad.py:1149: UserWarning: Use the 'save_best_solutions' parameter with caution as it may cause memory overflow when either the number of generations or number of genes is large.
  warnings.warn("Use the 'save_best_solutions' parameter with caution as it may cause memory overflow when either the number of genera

Generation :  1
The last best Solution :  {(-28.0, 6.0, 2.7, 328)}
Fitness of the last best solution : {0.166}
Generation :  2
The last best Solution :  {(-28.0, 9.5, 2.7, 328)}
Fitness of the last best solution : {0.186}
Generation :  3
The last best Solution :  {(-29.0, 9.5, 2.7, 141)}
Fitness of the last best solution : {0.188}
Generation :  4
The last best Solution :  {(-29.0, 9.5, 2.7, 141)}
Fitness of the last best solution : {0.218}
Generation :  5
The last best Solution :  {(-30.0, 9.5, 2.7, 141)}
Fitness of the last best solution : {0.218}
Generation :  6
The last best Solution :  {(-30.0, 9.5, 2.7, 141)}
Fitness of the last best solution : {0.226}
Generation :  7
The last best Solution :  {(-30.0, 9.5, 2.7, 115)}
Fitness of the last best solution : {0.226}
Generation :  8
The last best Solution :  {(-30.0, 9.4, 2.7, 115)}
Fitness of the last best solution : {0.227}
Generation :  9
The last best Solution :  {(-30.0, 9.4, 1.9, 115)}
Fitness of the last best solution : {0.228}
G

In [29]:
Evo_Data, Evo_Data_Full = GA_Opt.save_results(filefold=Mixing_Path, genre_columns=['Thre','Ratio','Attk','Release'], is_outputfulldata = True)

In [ ]:
Evo_Data

In [ ]:
Evo_Data_Full

### Import MC Optimiser

In [22]:
rng = np.random.default_rng(0)
rng

Generator(PCG64) at 0x7F322C7E5000

In [27]:
np.round(rng.uniform([-30.0,  1.0,  1.0, 100.0], [  0.0, 10.0, 20.0, 500.0], size=4),2)

array([ -4.1 ,   5.87,   6.69, 269.07])